# Exploring the t5 model for text detoxification



In [2]:
# installing huggingface libraries for dataset, models and metrics
!pip install datasets transformers[sentencepiece] sacrebleu

!pip install numpy==1.24.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 103.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 92.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23

In [3]:
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 6.4 MB/s eta 0:00:00


In [4]:
# Necessary inputs
import warnings

from datasets import load_dataset, load_metric
import transformers
import datasets
import random
import pandas as pd
import numpy as np
import torch
from IPython.display import display, HTML

warnings.filterwarnings('ignore')

## Selecting the model
More info about the model can be found  [here](https://huggingface.co/s-nlp/bart-base-detox).

In [7]:
# selecting model checkpoint
from transformers import BartForConditionalGeneration, AutoTokenizer

base_model_name = 'facebook/bart-base'
model_name = 'SkolkovoInstitute/bart-base-detox'
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

## Loading the dataset

In [8]:
# setting random seed for transformers library
transformers.set_seed(42)

# Load the BLUE metric
metric = load_metric("sacrebleu")

In [9]:
df = pd.read_csv("processed.tsv", sep ="\t")
df = df.rename(columns={"translation": "target", "reference": "text"})
df.head(5)

,text,target,similarity,lenght_diff,ref_tox,trn_tox
0,I've spent the last seven years with four smel...,I've spent the last seven years with four men.,0.950000,0.129630,0.990042,0.000041
1,tell me something I don't fucking know.,Tell me something I don't know!,0.949999,0.200000,0.971418,0.000079
2,you're such a fucking tough guy.,You're such a Tough guy.,0.949997,0.242424,0.977852,0.000238
3,someone tried to kill her with an injection of...,Someone tried to kill her by injecting the virus?,0.949996,0.137931,0.980873,0.102180
4,I could look at women's legs for hours.,I could look at a woman's legs for hours.,0.949996,0.047619,0.941838,0.006438


## Dataset
Downloaded from HuggingFace dataset is a `DatasetDict`. It contains keys `["train", "validation", "test"]` - which represents a dataset splits

In [10]:
train, validate, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])

In [11]:
train_dataset = datasets.Dataset.from_dict(train)
test_dataset = datasets.Dataset.from_dict(test)
validation_dataset = datasets.Dataset.from_dict(validate)

In [12]:
raw_datasets = datasets.DatasetDict({"train":train_dataset,"test":test_dataset, "validation": validation_dataset})

## Preprocessing the data
As usual we will need to preprocess data and tokenize it before passing to model

In [13]:
# prefix for model input
prefix = "Detoxify this sentence:"

In [14]:
max_input_length = 128
max_target_length = 128
source_lang = "toxic"
target_lang = "detoxified"
padding = "max_length"

def preprocess_function(examples):
    inputs = [prefix + ex for ex in examples["text"]]
    targets = [ex for ex in examples["target"]]

    model_inputs = tokenizer(inputs, max_length=max_input_length, padding='longest')
    print(model_inputs)
    # Setup the tokenizer for targets
    labels = tokenizer(targets, max_length=max_target_length, padding='longest')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["target"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [15]:
# example of preprocessing
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[0, 43170, 4325, 4591, 42, 3645, 35, 7682, 693, 11, 886, 4265, 734, 79, 18, 1169, 350, 5886, 50, 350, 7174, 50, 350, 402, 4, 2], [0, 43170, 4325, 4591, 42, 3645, 35, 32917, 6, 492, 162, 110, 2335, 6, 8, 38, 581, 4309, 24, 4, 370, 214, 15092, 4, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[0, 22466, 693, 11, 886, 4265, 79, 18, 350, 5886, 50, 350, 22877, 4, 2], [0, 8458, 225, 6, 492, 162, 110, 2335, 6, 8, 38, 581, 4309, 24, 4, 2]]}

In [16]:
cropped_datasets = raw_datasets
tokenized_datasets = cropped_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [17]:
type(tokenized_datasets["train"][:2]['labels'][0][0])

int

## Fine-tuning the model

In [19]:
from transformers import AutoModel, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoConfig

In [26]:
# defining the parameters for training
batch_size = 32
model_name = base_model_name.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-detoxify",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    report_to='tensorboard',
)

In [21]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [22]:
import numpy as np

# simple postprocessing for text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

# compute metrics function to pass to trainer
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [27]:
# instead of writing train loop we will use Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [28]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.944500,0.974032,44.625100,12.542500
2,0.942000,0.943298,45.135900,12.551500
3,0.864400,0.939642,45.249000,12.551600


TrainOutput(global_step=2814, training_loss=0.9367687059956912, metrics={'train_runtime': 1330.049, 'train_samples_per_second': 67.667, 'train_steps_per_second': 2.116, 'total_flos': 2336462472314880.0, 'train_loss': 0.9367687059956912, 'epoch': 3.0})

In [29]:
# saving model
trainer.save_model('best')

In [30]:
from transformers import AutoModelForSeq2SeqLM
# loading the model and run inference for it
model = AutoModelForSeq2SeqLM.from_pretrained('best')
model.eval()
model.config.use_cache = False

In [39]:
def translate(model, inference_request, tokenizer=tokenizer):
    input_ids = tokenizer(inference_request, return_tensors="pt").input_ids
    outputs = model.generate(input_ids=input_ids)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True,temperature=0))
    return tokenizer.decode(outputs[0], skip_special_tokens=True,temperature=0)

In [40]:
inference_request = prefix + 'You are a fucking piece of shit'
translate(model, inference_request,tokenizer)

you're a piece of shit.


"you're a piece of shit."

In [41]:
inference_request = prefix + "kill yourself"
translate(model, inference_request,tokenizer)

Kill yourself.


'Kill yourself.'

In [48]:
inference_request = prefix + "You are a faggot"
translate(model, inference_request,tokenizer)

you're gay.


"you're gay."

This is just because I'm using a google collab

In [42]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [35]:
!zip -r /content/best_SKOLKOVO.zip /content/best

  adding: content/best/ (stored 0%)
  adding: content/best/tokenizer_config.json (deflated 76%)
  adding: content/best/model.safetensors (deflated 8%)
  adding: content/best/special_tokens_map.json (deflated 52%)
  adding: content/best/training_args.bin (deflated 50%)
  adding: content/best/vocab.json (deflated 59%)
  adding: content/best/merges.txt (deflated 53%)
  adding: content/best/generation_config.json (deflated 41%)
  adding: content/best/config.json (deflated 63%)
  adding: content/best/tokenizer.json (deflated 72%)


# Inference. Analysis


In [43]:

from transformers import RobertaForSequenceClassification, RobertaTokenizer
clf_name = 'SkolkovoInstitute/roberta_toxicity_classifier_v1'
clf = RobertaForSequenceClassification.from_pretrained(clf_name).to(device);
clf_tokenizer = RobertaTokenizer.from_pretrained(clf_name)

Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier_v1 were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [44]:
def predict_toxicity(texts):
    with torch.inference_mode():
        inputs = clf_tokenizer(texts, return_tensors='pt', padding=True).to(clf.device)
        out = torch.softmax(clf(**inputs).logits, -1)[:, 1].cpu().numpy()
    return out

In [45]:
predict_toxicity(['hello world', 'hello aussie', 'hello fucking bitch'])

array([5.052518e-05, 8.788534e-05, 9.996809e-01], dtype=float32)

In [46]:
inference_request = 'This model is not fucking working'

predict_toxicity([inference_request, translate(model, prefix + inference_request,tokenizer)])

this model is not working.


array([9.7460073e-01, 4.3353422e-05], dtype=float32)